In [5]:
# Imports
import base64
import io
import zlib
from datetime import datetime
import sys
sys.path.append(".")
from src import configs

In [6]:
def decode_grib_file(encoded_data, output_path):
    """
    Decodes a base64 encoded and zlib compressed GRIB file and saves it to a specified location.

    Args:
    - encoded_data (str): The base64 encoded string of the zlib compressed GRIB file content.
    - output_path (str): The path where the decoded GRIB file should be saved.

    Returns:
    - str: Path to the saved file.
    """

    # Split the received data into lines and extract only the message content lines, skipping the order numbers
    lines = encoded_data.strip().split("\n")
    encoded_data_without_nrs = ''.join([lines[i] for i in range(len(lines)) if i % 3 == 1])

    # Decode the base64 encoded data
    zipped_data = base64.b64decode(encoded_data_without_nrs)

    # Decompress the zlib data to obtain the original GRIB file
    grib_binary = zlib.decompress(zipped_data)

    # Save the decoded data to the specified path
    with open(output_path, 'wb') as file:
        file.write(grib_binary)

    return output_path

In [7]:
# Copy all messages received from the inreach
TEXT_RECIEVED = """
0
eJxzD/J0YmDoYWRgkGFKmvW/QTGTgUucQ5yBgZFBhYuBgUEUxG
0
1
JQYPjPwMDBwM4Qe6BR0qGBoWdBw6MJ7BfYLzgwgIELBwPDofjwHA4jBp
1
2
+ApPjQcHvrxIDEvKaaeNewjPyyKTNi0vIyUksnTefm4RVT80ivlFV
2
3
UU7Vxjo/R09E31ZBRcmcwBwJ33G4xINEtZRuyODJMGFTSi8tTI5w
3
4
dQrJq+6dFBWckl7ZPmGiYWVbbWtEy0UBLzdA1sazGxsHKUNXQMdLH
4
5
zljfwMhIHatblMgPl4Jtkzg8ShZMbOhqLw1v6qkqbOuoT0quyE6
5
6
IKe1MSMqLd/VNLDDQM3WxNrN1N7e2NNOU13XwcbJw0JNk0CTkFlLD
6
7
JVW/isNMs6U1t7CgODAmq6A0sdwnOi48NT48JFMzLj3UN8In2Nw5
7
8
wM3C1N7Czy/Y00WS38I3OtrBWIGPAewWADeQjU4=
8
"""

# Define the path where the GRIB file should be saved
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
OUTPUT_PATH = configs.OUTPUT_DIR + f'decoded_grib_{current_time}.grb'

# Use the function to decode and save the GRIB file
df = decode_grib_file(TEXT_RECIEVED, OUTPUT_PATH)